In [6]:
import logging
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_link(self, switch, device):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_link, switch, device)
            for record in result:
                print("Created friendship between: {p1}, {p2}".format(
                    p1=record['p1'], p2=record['p2']))

    @staticmethod
    def _create_and_return_link(tx, switch, device):

        # To learn more about the Cypher syntax,
        # see https://neo4j.com/docs/cypher-manual/current/

        # The Reference Card is also a good resource for keywords,
        # see https://neo4j.com/docs/cypher-refcard/current/

        query = (
            "CREATE (p1:Switch { name: $switchname }) "
            "CREATE (p2:Device { name: $flogi }) "
            "CREATE (p1)-[:FLOGI]->(p2) "
            "RETURN p1, p2"
        )
        result = tx.run(query, switchname=swittchname, flogi=flogi)
        try:
            return [{"p1": record["p1"]["name"], "p2": record["p2"]["name"]}
                    for record in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_switch(self, switchname):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_switch, switchname)
            for record in result:
                print("Found switch: {record}".format(record=record))

    @staticmethod
    def _find_and_return_switch(tx, person_name):
        query = (
            "MATCH (p:Switch) "
            "WHERE p.name = $switchname "
            "RETURN p.name AS switchname"
        )
        result = tx.run(query, switchname=switchname)
        return [record["name"] for record in result]

if __name__ == "__main__":
    # See https://neo4j.com/developer/aura-connect-driver/ for Aura specific connection URL.
    scheme = "neo4j"  # Connecting to Aura, use the "neo4j+s" URI scheme
    host_name = "localhost"
    port = 7687
    url = "{scheme}://{host_name}:{port}".format(scheme=scheme, host_name=host_name, port=port)
    user = "admin"
    password = "admin"
    app = App(url, user, password)
    app.create_link(switchname, device)
    app.find_switch(switchname)
    app.close()

Created friendship between: Alice, David
Found person: Alice
Found person: Alice
